Neural Networks http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [5]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120) # can we just set next layer size?
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        self.name = 'conv-pool-conv-pool-MLP'
        
    def forward(self, x): # input: nSamples x nChannels x Height x Width
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x)) # first dim -1
        x = F.relu(self.fc1(x)) # relu(Wx+b)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)
print(net.name)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
conv-pool-conv-pool-MLP


In [7]:
params = list(net.parameters())
print(len(params))
print([x.size() for x in params])

10
[torch.Size([6, 1, 5, 5]), torch.Size([6]), torch.Size([16, 6, 5, 5]), torch.Size([16]), torch.Size([120, 400]), torch.Size([120]), torch.Size([84, 120]), torch.Size([84]), torch.Size([10, 84]), torch.Size([10])]


In [26]:
input = Variable(torch.randn(1,32,32))
input = input.unsqueeze(0)
out = net(input)
print(out.size())
print(out)

torch.Size([1, 10])
Variable containing:
-0.0361  0.0345  0.0642  0.0409  0.0526 -0.1005  0.0972  0.1983  0.0663 -0.0567
[torch.FloatTensor of size 1x10]



In [11]:
net.zero_grad()
out.backward(torch.randn(1,10))

`torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, `nn.Conv2d `will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.

If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.

In [19]:
target = Variable(torch.arange(1,11))
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(out,target)
print(loss)

Variable containing:
 38.4953
[torch.FloatTensor of size 1]



In [22]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions)

((<AddmmBackward object at 0x10ec73240>, 0),)
((<ExpandBackward object at 0x10ec69128>, 0), (<ThresholdBackward object at 0x10ec69198>, 0), (<TBackward object at 0x10ec697b8>, 0))


In [23]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

Variable containing:
 0.0324
-0.0178
 0.1160
-0.0043
 0.0469
 0.0625
[torch.FloatTensor of size 6]



In [24]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*lr)

In [27]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update